In [2]:
import numpy as np
np.random.multinomial??

In [12]:
with open('data/nietzsche.txt') as f:
    text = f.read().lower()
print ''.join(open('data/nietzsche.txt').readlines()[:100])

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to Truth, have been unskilled and unseemly methods for
winning a woman? Certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--IF,
indeed, it stands at all! For there are scoffers who maintain that it
has fallen, that all dogma lies on the ground--nay more, that it is at
its last gasp. But to speak seriously, there are good grounds for hoping
that all dogmatizing in philosophy, whatever solemn, whatever conclusive
and decided airs it has assumed, may have been only a noble puerilism
and tyronism; and probably the time is at hand when it will be once
and again understood WHAT has actually sufficed for the basis of such
imposing and abso

In [34]:
# 1.产生训练用的corpus, 对其中每个字符进行one-hot
max_len = 60
step = 3  # 每隔3个字符产生一个新的sequence
sentences = []
next_chars = []
print 'number of characters in corpus :',len(text) # 共600901个字符

number of characters in corpus : 600901


In [35]:
# 划分x和y
for i in range(0,len(text)-max_len,step): # 每隔3个字符产生一个新的sequence
    sentences.append(text[i:i+max_len])
    next_chars.append(text[i+max_len])
print 'extract ',len(sentences),' sentences from corpus'

extract  200281  sentences from corpus


In [36]:
# 用set集合找出文本中使用了那些字符: 包括字母, 标点, 换行符等
individual_characters = set(text) 
# chars=[char1,char2 ...]
chars = sorted(list(individual_characters))
print 'first 15 characters : ',chars[:15]
print 'Unique characters : ', len(chars)
# char_idxs={char:index}
char_idxs = dict((char,chars.index(char)) for char in chars)

first 15 characters :  ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4']
Unique characters :  59


In [37]:
# 对所有sentences进行one-hot编码
x = np.zeros((len(sentences),max_len,len(chars)))
y = np.zeros((len(sentences),len(chars)))
for i,sentence in enumerate(sentences):
    for j,char in enumerate(sentence):
        x[i,j,char_idxs[char]] = 1
    next_char = next_chars[i]
    y[i,char_idxs[next_char]] = 1

In [40]:
### 2.构建单层LSTM+softmax输出
from keras import layers,models
model = models.Sequential() 
model.add(layers.LSTM(128,input_shape=(max_len,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
model.fit(x,y,epochs=6,batch_size=32)

Epoch 1/6
200281/200281 [==============================] - 502s 3ms/step - loss: 2.2616
Epoch 2/6
200281/200281 [==============================] - 509s 3ms/step - loss: 1.8885
Epoch 3/6
200281/200281 [==============================] - 506s 3ms/step - loss: 1.7436
Epoch 4/6
200281/200281 [==============================] - 513s 3ms/step - loss: 1.6597
Epoch 5/6
200281/200281 [==============================] - 512s 3ms/step - loss: 1.6042
Epoch 6/6
200281/200281 [==============================] - 505s 3ms/step - loss: 1.5618


In [31]:
### 3. 从输出的softmax概率中抽样出character
def sample(distribute,temperature=1.0):
    distribute = np.array(distribute,dtype='float64')
    distribute = np.log(distribute) / temperature
    exp_distribute = np.exp(distribute)
    distribute = exp_distribute / np.sum(exp_distribute)
    multinomial = np.random.multinomial(1,distribute,1)
    return np.argmax(multinomial)

print sample([0.1,0.5,0.4])

1


In [68]:
### 4.产生400个字符的文章
generated_text = text[10:max_len]  # 使用corpus中的语句产生第一个预测的字符序列
sample_onehot = x[0,:,:]
for temperature in [0.2,0.5,1.0,1.2]:
    for i in range(400):
        sampled = np.array([sample_onehot])
        softmax_distribute = model.predict(sampled,verbose=0)[0]
        next_index = sample(softmax_distribute,temperature)
        next_char = chars[next_index]
        generated_text += next_char
        # 产生新的预测集
        next_char_onehot = np.zeros((1,len(chars)))
        next_char_onehot[0,next_index] = 1
        sample_onehot = np.concatenate((sample_onehot[1:,:],next_char_onehot),axis=0)
    print 'temperature:',temperature
    print generated_text
    print '-------------------------'


temperature: 0.2
supposing that truth is a woman--what then? is the most personal that it is the most feelings of the actions of the present of the sense of the person which is the sense of the sense of the sense of the seet of the sense of the sense of such a man of the sense of a sense of the enemorious and such and the soul and the person it is the soul and the sign in the most feelings of the problement of the sense of a self-contradicty of the and some such 
-------------------------
temperature: 0.5
supposing that truth is a woman--what then? is the most personal that it is the most feelings of the actions of the present of the sense of the person which is the sense of the sense of the sense of the seet of the sense of the sense of such a man of the sense of a sense of the enemorious and such and the soul and the person it is the soul and the sign in the most feelings of the problement of the sense of a self-contradicty of the and some such a man most bene and as its art of such 